# Finalise facebook posts
Finalises the facebook data.
And persists it to s3.
See docs for more information.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import pandas as pd

from phoenix import common
from phoenix.common import artifacts, utils
from phoenix.tag import export
from phoenix.tag import finalise
from phoenix.tag import object_filters

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"
TENANT_ID = None

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_posts"

# OUTPUT
FINAL_URL_FACEBOOK_POSTS = None
INCLUDE_OBJECTS_TENSIONS = True
INCLUDE_SENTIMENT = True

In [ ]:
run_params = common.run_params.general.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME)
    
url_config = {
    "YEAR_FILTER": YEAR_FILTER,
    "MONTH_FILTER": MONTH_FILTER,
    "OBJECT_TYPE": OBJECT_TYPE
}

TAGGING_RUNS_URL_FACEBOOK_POSTS_PULLED = run_params.art_url_reg.get_url("tagging_runs-facebook_posts_pulled", url_config)
TAGGING_RUNS_URL_PIPELINE_BASE = run_params.art_url_reg.get_url("tagging_runs-pipeline_base", url_config)
TAGGING_RUNS_URL_OBJECTS_TENSIONS = run_params.art_url_reg.get_url("tagging_runs-objects_tensions", url_config)
TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS = run_params.art_url_reg.get_url("tagging_runs-language_sentiment_objects", url_config)
TAGGING_RUNS_URL_FACEBOOK_POSTS_FINAL = run_params.art_url_reg.get_url("tagging_runs-facebook_posts_final", url_config)
TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE = run_params.art_url_reg.get_url("tagging_runs-facebook_posts_to_scrape", url_config)
TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE_DASHBOARD = run_params.art_url_reg.get_url("tagging_runs-facebook_posts_to_scrape_dashboard", url_config)
if not FINAL_URL_FACEBOOK_POSTS:
    FINAL_URL_FACEBOOK_POSTS = run_params.art_url_reg.get_url("final-facebook_posts", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_FACEBOOK_POSTS_PULLED,
TAGGING_RUNS_URL_PIPELINE_BASE,
TAGGING_RUNS_URL_OBJECTS_TENSIONS,
TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS,
TAGGING_RUNS_URL_FACEBOOK_POSTS_FINAL,
TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE,
TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE_DASHBOARD,
FINAL_URL_FACEBOOK_POSTS,
run_params.run_dt.dt,
sep='\n',
)

In [ ]:
# This dataframe is always needed for the finalise
facebook_posts_df = artifacts.dataframes.get(TAGGING_RUNS_URL_FACEBOOK_POSTS_PULLED).dataframe

In [ ]:
facebook_posts_df.head()

In [ ]:
objects = None
if INCLUDE_OBJECTS_TENSIONS:
    objects = artifacts.dataframes.get(TAGGING_RUNS_URL_OBJECTS_TENSIONS)
    objects.head()

In [ ]:
language_sentiment_objects = None
if INCLUDE_SENTIMENT:
    language_sentiment_objects = artifacts.dataframes.get(TAGGING_RUNS_URL_LANGUAGE_SENTIMENT_OBJECTS)
    language_sentiment_objects.head()

In [ ]:
facebook_posts_final = finalise.join_objects_to_facebook_posts(facebook_posts=facebook_posts_df, objects=objects, language_sentiment_objects=language_sentiment_objects)

In [ ]:
facebook_posts_final.head()

In [ ]:
facebook_posts_final.dtypes

In [ ]:
if "is_key_object" in facebook_posts_final.columns:
    posts_to_scrape = export.get_posts_to_scrape(object_filters.get_key_objects(facebook_posts_final))
    export.persist_posts_to_scrape(posts_to_scrape, TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE, TAGGING_RUNS_URL_FACEBOOK_POSTS_TO_SCRAPE_DASHBOARD)

In [ ]:
_ = artifacts.dataframes.persist(FINAL_URL_FACEBOOK_POSTS, facebook_posts_final)

In [ ]:
_ = artifacts.dataframes.persist(TAGGING_RUNS_URL_FACEBOOK_POSTS_FINAL, facebook_posts_final)

In [ ]:
artifacts.dataframes.read_schema(TAGGING_RUNS_URL_FACEBOOK_POSTS_FINAL)